In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import os 
import sys
from tqdm import tqdm
sys.path.insert(0, os.getcwd() + '/code')
from viz import *
from helper_functions import * 

In [3]:
df_game = pd.read_csv("data/games.csv")
df_tracking = pd.read_csv("data/week1.csv")
df_tracking = compute_orientation(df_tracking)
df_play = pd.read_csv("data/plays.csv")
df_players = pd.read_csv("data/players.csv")
df_pffScoutingData = pd.read_csv("data/pffScoutingData.csv")

In [11]:
animate_play(df_tracking,df_play,df_players,df_pffScoutingData,2021090900, 2330, displayZone=True, displayOrientations=False).show()

In [ ]:
gameId = 2021090900
playId = 97

playIds = df_play[df_play.gameId==gameId].playId.unique()
playIds.sort()
for playId in playIds[:5]:
    selected_play_df = df_play[(df_play.playId==playId)&(df_play.gameId==gameId)].copy()    
    tracking_players_df = pd.merge(df_tracking,df_players,how="left",on = "nflId")
    tracking_players_df = pd.merge(tracking_players_df,df_pffScoutingData,how="left",on = ["nflId","playId","gameId"])
    selected_tracking_df = tracking_players_df[(tracking_players_df.playId==playId)&(tracking_players_df.gameId==gameId)].copy()
    selected_tracking_df = beaten_by_defender(gameId, playId, df_pffScoutingData, selected_tracking_df, seuil = 0.5)


    sorted_frame_list = selected_tracking_df.frameId.unique()
    sorted_frame_list.sort()
    if selected_play_df.empty :
        print(playId)
        continue
    line_of_scrimmage = selected_play_df.absoluteYardlineNumber.values[0]
    playDescription = selected_play_df.playDescription.values[0]
    print(f" PlayId : {playId} - {playDescription}")
    aire_t = list()
    if 'pass_forward' in selected_tracking_df.event.unique():
        eventFrame = selected_tracking_df[selected_tracking_df.event == 'pass_forward'].iloc[0].frameId
    else :
        eventFrame = 1000
    for frameId in sorted_frame_list:
        #if eventFrame == frameId :
        #    break
        selected_frame_df = selected_tracking_df[selected_tracking_df.frameId == frameId]
        offensive_points = get_Oline_position(selected_frame_df)
        defensive_points = get_Dline_position(selected_frame_df)
        QB_zone = calculate_Oline_zones(offensive_points, line_of_scrimmage)
        region_polys, region_pts, players_points = calculate_voronoi_zones(QB_zone, offensive_points, defensive_points)
        aire_t.append(pocketArea(region_polys, region_pts, players_points))

    plt.plot(aire_t)
    plt.vlines(eventFrame, 0, np.max(aire_t), color = 'black', linestyles='--',)
    plt.show()
    

In [ ]:
gameId = 2021090900

playIds = df_play[df_play.gameId==gameId].playId.unique()
playIds.sort()
for playId in tqdm(playIds):
    selected_play_df = df_play[(df_play.playId==playId)&(df_play.gameId==gameId)].copy()  
    if 'SCRAMBLE' in selected_play_df.dropBackType.unique():  
        tracking_players_df = pd.merge(df_tracking,df_players,how="left",on = "nflId")
        tracking_players_df = pd.merge(tracking_players_df,df_pffScoutingData,how="left",on = ["nflId","playId","gameId"])
        selected_tracking_df = tracking_players_df[(tracking_players_df.playId==playId)&(tracking_players_df.gameId==gameId)].copy()
        selected_tracking_df = tracking_players_df[(tracking_players_df.playId==playId)&(tracking_players_df.gameId==gameId)].copy()
        selected_tracking_df = beaten_by_defender(gameId, playId, df_pffScoutingData, selected_tracking_df, seuil = 0.5)

        sorted_frame_list = selected_tracking_df.frameId.unique()
        sorted_frame_list.sort()
        if selected_play_df.empty :
            print(playId)
            continue
        line_of_scrimmage = selected_play_df.absoluteYardlineNumber.values[0]
        playDescription = selected_play_df.playDescription.values[0]
        print(f" PlayId : {playId} - {playDescription}")

        QB_dis_list = list(selected_tracking_df[selected_tracking_df.officialPosition == "QB"].dis)
        QB_dis_covered = [QB_dis_list[0]] + [np.sum(QB_dis_list[:k]) for k in range(1,len(QB_dis_list))]
        aire_t = list()

        for frameId in sorted_frame_list:
            #if eventFrame == frameId :
            #    break
            selected_frame_df = selected_tracking_df[selected_tracking_df.frameId == frameId]
            offensive_points = get_Oline_position(selected_frame_df)
            defensive_points = get_Dline_position(selected_frame_df)
            QB_zone = calculate_Oline_zones(offensive_points, line_of_scrimmage)
            region_polys, region_pts, players_points = calculate_voronoi_zones(QB_zone, offensive_points, defensive_points)
            aire_t.append(pocketArea(region_polys, region_pts, players_points))
        fig, ax = plt.subplots(2,1, figsize=(7,14))
        ax[0].plot(sorted_frame_list, aire_t)
        
        if 'pass_forward' in selected_tracking_df.event.unique():
            eventFrame = selected_tracking_df[selected_tracking_df.event == 'pass_forward'].iloc[0].frameId
            ax[0].vlines(eventFrame, 0, np.max(aire_t), color = 'black', linestyles='--',)
        else :
            eventFrame = 1000
        ax[1].plot(sorted_frame_list, QB_dis_covered, color = 'red')
        plt.show()
    

In [ ]:
print(f'Total n° of plays : {len(df_play)}')
scramble_mask = "'SCRAMBLE' in dropBackType | 'SCRAMBLE_ROLLOUT_RIGHT' in dropBackType | 'SCRAMBLE_ROLLOUT_LEFT' in dropBackType"
scramble_plays = df_play.query(scramble_mask)#[["gameId","playId","dropBackType"]]
print(f'n° of scramble plays : {len(scramble_plays)}')
for scramble_id in tqdm(range(len(scramble_plays))):

    scramble_play = scramble_plays.iloc[scramble_id].playId
    scramble_game = scramble_plays.iloc[scramble_id].gameId

    selected_play_df = df_play[(df_play.playId==scramble_play)&(df_play.gameId==scramble_game)].copy()  
    tracking_players_df = pd.merge(df_tracking,df_players,how="left",on = "nflId")
    tracking_players_df = pd.merge(tracking_players_df,df_pffScoutingData,how="left",on = ["nflId","playId","gameId"])
    selected_tracking_df = tracking_players_df[(tracking_players_df.playId==scramble_play)&(tracking_players_df.gameId==scramble_game)].copy()
    selected_tracking_df = beaten_by_defender(scramble_game, scramble_play , df_pffScoutingData, selected_tracking_df, seuil = 0.5)
    selected_tracking_df = scramble(scramble_game, scramble_play , df_pffScoutingData, selected_tracking_df, seuil = 0.5)
    sorted_frame_list = selected_tracking_df.frameId.unique()
    sorted_frame_list.sort()
    if selected_play_df.empty :
        print(playId)
        continue
    line_of_scrimmage = selected_play_df.absoluteYardlineNumber.values[0]
    playDescription = selected_play_df.playDescription.values[0]
    print(f" PlayId : {scramble_game} - {scramble_play} - {playDescription}")

    QB_dis_list = list(selected_tracking_df[selected_tracking_df.officialPosition == "QB"].dis)
    QB_dis_covered = [QB_dis_list[0]] + [np.sum(QB_dis_list[:k]) for k in range(1,len(QB_dis_list))]
    aire_t = list()

    for frameId in sorted_frame_list:
        #if eventFrame == frameId :
        #    break
        selected_frame_df = selected_tracking_df[selected_tracking_df.frameId == frameId]
        offensive_points = get_Oline_position(selected_frame_df)
        defensive_points = get_Dline_position(selected_frame_df)
        QB_zone = calculate_Oline_zones(offensive_points, line_of_scrimmage)
        region_polys, region_pts, players_points = calculate_voronoi_zones(QB_zone, offensive_points, defensive_points)
        aire_t.append(pocketArea(region_polys, region_pts, players_points))
    aire_min_during_scramble = np.min(aire_t)
    scramble_plays.loc[scramble_id, 'scrambleArea'] = aire_min_during_scramble
#print(scramble_plays[['gameId','playId','scrambleArea']])

    plt.plot(sorted_frame_list, aire_t)

    if 'pass_forward' in selected_tracking_df.event.unique():
        eventFrame = selected_tracking_df[selected_tracking_df.event == 'pass_forward'].iloc[0].frameId
        plt.vlines(eventFrame, 0, np.max(aire_t), color = 'black', linestyles='--',)
    else :
        eventFrame = 1000
    plt.show()


In [7]:
from multiprocessing import Pool

df_area = pd.read_csv("data/area/week1.csv")
playIds = df_area.playId.unique()

#play_list = list()
#game_list = list()
#event_list = list()
#te_list = list()
#Ae_list = list() 
#tc_list = list()
#Ac_list = list()

def features_one_play(playId) : 
    gameId = df_area[df_area.playId == playId].gameId.unique()[0]
    selected_area_df = df_area[(df_area.playId==playId)&(df_area.gameId==gameId)].copy() 
    selected_play_df = df_play[(df_play.playId==playId)&(df_play.gameId==gameId)].copy()  
    tracking_players_df = pd.merge(df_tracking,df_players,how="left",on = "nflId")
    tracking_players_df = pd.merge(tracking_players_df,df_pffScoutingData,how="left",on = ["nflId","playId","gameId"])
    selected_tracking_df = tracking_players_df[(tracking_players_df.playId==playId)&(tracking_players_df.gameId==gameId)].copy()
    selected_tracking_df = beaten_by_defender(gameId, playId , df_pffScoutingData, selected_tracking_df, seuil = 0.5)
    selected_tracking_df = scramble(gameId, playId, df_pffScoutingData, selected_tracking_df, seuil = 0.5)
    event, te = compute_t_event(gameId, playId, selected_play_df, df_pffScoutingData, selected_tracking_df)
    Ae = selected_area_df[selected_area_df.frameId == te].Area.iloc[0]
    Ac = np.max(selected_area_df.Area)
    tc = selected_area_df[selected_area_df.Area == Ac].frameId.iloc[0]
    return pd.DataFrame([[playId, gameId, event, te, Ae, tc, Ac]], 
                                        columns = ['playId', 'gameId', 'event', 'te', 'Ae', 'tc', 'Ac'])
    #play_list.append(playId)
    #game_list.append(gameId)
    #event_list.append(event)
    #te_list.append(te)
    #Ae_list.append(Ae)
    #tc_list.append(tc)
    #Ac_list.append(Ac)

with Pool() as mp_pool:
    Area_features = mp_pool.map(features_one_play, playIds[:20])

Area_features = pd.concat(Area_features)

print(Area_features)



  0%|          | 0/5 [00:00<?, ?it/s]

97
2021090900


 20%|██        | 1/5 [00:02<00:10,  2.74s/it]

137
2021090900


 40%|████      | 2/5 [00:05<00:07,  2.61s/it]

...

282
2021090900


 80%|████████  | 4/5 [00:10<00:02,  2.74s/it]

349
2021090900


100%|██████████| 5/5 [00:13<00:00,  2.66s/it]

   playId      gameId event  te         Ae  tc         Ac
0      97  2021090900  pass  40  12.237518  18  29.001750
1     137  2021090900  pass  32  16.305513  19  27.422800
2     187  2021090900  pass  28  18.312900  20  25.935250
3     282  2021090900  pass  38  29.901092  27  40.623989
4     349  2021090900  pass  33  14.784734  19  24.298700
